In [2]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [18]:
gps_data = pd.read_csv("utiaswinter_trial1/teach/gps.csv")

In [19]:
# convert unix timestamps into flight time format in seconds (start time 0 seconds)
gps_data['Timestamp']/1e9 - (gps_data['Timestamp']/1e9).iloc[0]

0          0.000000
1          0.042109
2          0.079250
3          0.100368
4          0.121965
            ...    
14982    309.246205
14983    309.304385
14984    309.324149
14985    309.345286
14986    309.364764
Name: Timestamp, Length: 14987, dtype: float64

In [20]:
gps_data

,Timestamp,latitude,longitude,altitude
0,1.604610e+18,43.781741,-79.466295,157.827347
1,1.604610e+18,43.781741,-79.466295,157.827881
2,1.604610e+18,43.781741,-79.466295,157.828613
3,1.604610e+18,43.781741,-79.466295,157.829041
4,1.604610e+18,43.781741,-79.466295,157.829361
...,...,...,...,...
14982,1.604610e+18,43.781705,-79.466206,157.730148
14983,1.604610e+18,43.781705,-79.466206,157.729050
14984,1.604610e+18,43.781705,-79.466206,157.728577
14985,1.604610e+18,43.781705,-79.466206,157.728165
